In [18]:
import numpy as np
import dataFlux
import matplotlib.pyplot as plt
import pyspedas
import PyGeopack as gp
import TraceFieldMy as TFm
from scipy.interpolate import interp1d

RE = 6371

In [5]:
data = dataFlux.get_dataflux(4, "20120907", "20130901")

In [8]:
data.keys()

dict_keys(['times', 'MLAT1', 'MLAT2', 'Lshell', 'fluxes1', 'fluxes2', 'fluxesRBSP', 'Pdyn', 'SymH', 'By', 'Bz', 'kp'])

In [68]:
f1 = data.get('fluxes1')
f1[np.where(f1 < 0)] = 0
f1 = np.sum(f1, axis=1)

f2 = data.get('fluxes2')
f2[np.where(f2 < 0)] = 0
f2 = np.sum(f2, axis=1)

ft = data.get('fluxesRBSP')
ft[np.where(ft < 0)] = 0
ft = np.sum(ft, axis=1)

In [ ]:
f1[535,:]

In [10]:
bins = np.logspace(2, 8, 40)
H, xedges, yedges = np.histogram2d(f1, f2, bins=bins)

In [ ]:
np.savetxt("binsX.txt",xedges)
np.savetxt("binsY.txt",yedges)
np.savetxt("binsZ.txt",H)

In [ ]:
np.max(f2)

In [ ]:
f1 = data.get('fluxesRBSP')

In [ ]:
def interp1(x,y,xp):
    inds = np.logical_not(np.isnan(y))
    x = x[inds]
    y = y[inds]
    f = interp1d(x,y,bounds_error=False,fill_value=np.nan)
    return f(xp)



In [78]:
mageis_vars = pyspedas.rbsp.mageis(["20140112", "20140113"], probe='a', level='l3', rel='rel04', notplot=True)
times = np.array(mageis_vars['FEDU']['x'])
fluxes0 = np.array(mageis_vars['FEDU']['y'])
alpha0 = np.array(mageis_vars['FEDU']['v1'])
energy = np.array(mageis_vars['FEDU']['v2'])

inds = np.where(np.isnan(fluxes0[:,-1,:]))
fluxes0[(inds[0],-1*np.ones_like(inds[0]),inds[1])] = fluxes0[(inds[0],np.zeros_like(inds[0]),inds[1])]

23-Dec-21 04:54:38: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/rbsp/rbspa/l3/ect/mageis/sectors/rel04/2014/
23-Dec-21 04:54:39: File is current: D:\SPEDAS files\rbsp\rbspa/l3/ect/mageis/sectors/rel04/2014/rbspa_rel04_ect-mageis-l3_20140112_v8.1.0.cdf


In [110]:
alpha = np.linspace(0, 180, 181)
alpha = alpha[:-1] + np.diff(alpha)/2

da0 = np.mean(np.diff(alpha0))
da = np.mean(np.diff(alpha))

alpha01 = np.concatenate(([0], alpha0, [180]))
fluxes01 = np.concatenate((fluxes0[:,[0],:], fluxes0, fluxes0[:,[-1],:]), axis=1)

f = interp1d(alpha01, fluxes01, kind='linear', axis=1)

In [111]:
gg = f(alpha) * da / da0


In [108]:
print(np.sum(gg))
print(np.sum(fluxes0))

nan
nan


(7900, 13, 25)

In [ ]:
mageis_vars = pyspedas.rbsp.mageis(["20140112", "20140113"], probe='a', level='l3', rel='rel04', notplot=True)
times = np.array(mageis_vars.get('FEDU').get('x'))
mageis_pos_geo = mageis_vars.get('Position').get('y')/RE

In [ ]:
mageis_pos_gsm = np.column_stack(gp.GEOtoGSE(*mageis_pos_geo.T, *TFm.times_to_dateUt(times)))

In [ ]:
emfisis_vars = pyspedas.rbsp.emfisis(["20140112", "20140113"], probe='a', level='l3', notplot=True, coord='gse')
emfisis_t = emfisis_vars.get('coordinates').get('x')
emfisis_pos = emfisis_vars.get('coordinates').get('y')/RE

In [ ]:
interp_posX = interp1(emfisis_t, emfisis_pos[:,0], times)
interp_posY = interp1(emfisis_t, emfisis_pos[:,1], times)
interp_posZ = interp1(emfisis_t, emfisis_pos[:,2], times)
interp_pos = np.column_stack((interp_posX, interp_posY, interp_posZ))


In [ ]:
print(np.mean(np.sum(np.sqrt((interp_pos - mageis_pos_gsm)**2), axis=1)))
print(np.max(np.sum(np.sqrt((interp_pos - mageis_pos_gsm)**2), axis=1)))

In [ ]:
print(mageis_pos_gsm[1])
print(emfisis_pos[1])

In [ ]:
np.max(np.abs(mageis_pos_gsm - interp_pos))

In [ ]:
TFm.times_to_dateUt(times)

In [ ]:
emfisis_vars = pyspedas.rbsp.emfisis(["20140112", "20140113"], probe='a', level='l3', notplot=True, coord='gsm')
emfisis_t1 = emfisis_vars.get('coordinates').get('x')
emfisis_pos_1 = emfisis_vars.get('coordinates').get('y')/RE

emfisis_vars = pyspedas.rbsp.emfisis(["20140112", "20140113"], probe='a', level='l3', notplot=True, coord='gse')
emfisis_t2 = emfisis_vars.get('coordinates').get('x')
emfisis_pos_2 = emfisis_vars.get('coordinates').get('y')/RE

emfisis_pos_2r = np.column_stack(gp.GSEtoGSM(*emfisis_pos_1.T, *TFm.times_to_dateUt(emfisis_t1)))
print(np.mean(np.sum(np.sqrt((emfisis_pos_2r - emfisis_pos_2)**2), axis=1)))
print(np.max(np.sum(np.sqrt((emfisis_pos_2r - emfisis_pos_2)**2), axis=1)))


In [ ]:
np.sum(emfisis_pos_2r**2, axis=1)

In [ ]:
def get_coord_GSM(date_start_str, date_end_str, times):
    emfisis_vars = pyspedas.rbsp.emfisis([date_start_str, date_end_str], probe='a', level='l3', notplot=True, coord='gsm')
    pos_times = emfisis_vars.get('coordinates').get('x')
    pos_coords = emfisis_vars.get('coordinates').get('y')
    if(np.nanmean(np.abs(pos_coords)) > 40):
        pos_coords = pos_coords / 6371.2
    x = pos_coords[:,0]
    y = pos_coords[:,1]
    z = pos_coords[:,2]

    X = interp1(pos_times, x, times)
    Y = interp1(pos_times, y, times)
    Z = interp1(pos_times, z, times)
    return np.column_stack((X,Y,Z))

In [ ]:
mageis_vars = pyspedas.rbsp.mageis(["20140112", "20140113"], probe='a', level='l3', rel='rel04', notplot=True)
times = np.array(mageis_vars.get('FEDU').get('x'))

pos_gsm = get_coord_GSM("20140112", "20140113", times)


In [ ]:
pos_gsm[434, 2] = np.nan



In [5]:
data = dataFlux.get_dataflux(4, "20140101", "20140102")

In [4]:
data.keys()
def get_energy_int(f):
    f[np.where(f < 0)] = 0
    return np.sum(f, axis=1)



In [114]:
f1 = data.get('fluxes1')
f2 = data.get('fluxes2')
ft = data.get('fluxesRBSP')

In [115]:
f1e = get_energy_int(f1)
f2e = get_energy_int(f2)
fte = get_energy_int(ft)

In [120]:
bins = np.logspace(2, 8, 200)
H, xedges, yedges = np.histogram2d(f1e, fte, bins=bins)

In [121]:
np.savetxt("binsX.txt",xedges)
np.savetxt("binsY.txt",yedges)
np.savetxt("binsZ.txt",H)

In [123]:
emfisis_vars = pyspedas.rbsp.emfisis(["20160623", "20160624"], probe='a', level='l3', notplot=True, coord='gsm')
mfisis_vars_coord = emfisis_vars.get('coordinates')

23-Dec-21 15:27:01: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/rbsp/rbspa/l3/emfisis/magnetometer/4sec/gsm/2016/
23-Dec-21 15:27:02: File is current: D:\SPEDAS files\rbsp\rbspa/l3/emfisis/magnetometer/4sec/gsm/2016/rbsp-a_magnetometer_4sec-gsm_emfisis-l3_20160623_v1.6.1.cdf


Value error for variable: Mag
Value error for variable: Magnitude
Value error for variable: delta
Value error for variable: lambda
Value error for variable: rms
Value error for variable: coordinates


In [130]:
if(np.array([]).size):
    print(1)
else:
    print(2)

2


In [11]:
import datetime

In [12]:
dtl  = (datetime.date(2012, 2, 12),)
dt = datetime.date(2012, 2, 12)
dt in dtl

True

In [13]:
dt.strftime('%Y%m%d')

'20120212'

In [31]:
date_start  = datetime.date(2012, 9, 7)
date_end    = datetime.date(2015, 1, 1)
date_current = date_start
date_step = datetime.timedelta(days=60)

In [32]:
R = 4
SymH = np.empty(shape=(0,))
while(date_current < date_end):
    date_start_p = date_current
    date_current += date_step
    date_end_p = min(date_current, date_end)
    out_data =  dataFlux.get_dataflux(R, date_start_p.strftime('%Y%m%d'), date_end_p.strftime('%Y%m%d'), days_step=20)
    # fluxes_data =  np.concatenate( (fluxes_data, get_energy_int(out_data.get('fluxes1'))) )
    SymH = np.concatenate( (SymH, out_data.get('SymH')) )
    print('Done: %s' % (date_current))


Done: 2012-11-06
Done: 2013-01-05
Done: 2013-03-06
Done: 2013-05-05
Done: 2013-07-04
Done: 2013-09-02
Done: 2013-11-01
Done: 2013-12-31
Done: 2014-03-01
Done: 2014-04-30
Done: 2014-06-29
Done: 2014-08-28
Done: 2014-10-27
Done: 2014-12-26
Done: 2015-02-24


In [25]:
SymHn = SymH[np.where(np.logical_not(np.isnan(SymH)))]

In [27]:
y, x =  np.histogram(SymHn, 100)

In [29]:
np.savetxt('datx.txt', x)
np.savetxt('daty.txt', y)

In [30]:
np.savetxt('symh.txt', SymHn)

In [10]:
data = dataFlux.get_dataflux(4, "20120907", "20150908")

In [5]:
f1 = data.get('fluxes1')
f2 = data.get('fluxes2')
ft = data.get('fluxesRBSP')
f1e = get_energy_int(f1)
f2e = get_energy_int(f2)
fte = get_energy_int(ft)

In [6]:
np.nanmax(f1e/fte)

23-Dec-21 23:14:03: C:\Users\PB720~1.SHU\AppData\Local\Temp/ipykernel_60720/957651613.py:1: RuntimeWarning: invalid value encountered in true_divide
  np.nanmax(f1e/fte)



1.0006599262757927

In [1]:
import numpy as np
import datetime
import os
import dataFlux
import fitFlows

Reading Data 100.00%


In [2]:
R = 4
date_start  = datetime.date(2012, 9, 7)
date_end    = datetime.date(2019, 10, 14)

date_current = date_start
date_step = datetime.timedelta(days=60)

fluxes1 = np.empty(shape=(0,25))
fluxes2 = np.empty(shape=(0,25))
SymH = np.empty(shape=(0,))
while(date_current < date_end):
    date_start_p = date_current
    date_current += date_step
    date_end_p = min(date_current, date_end)
    out_data = dataFlux.get_dataflux(R, date_start_p.strftime('%Y%m%d'), date_end_p.strftime('%Y%m%d'), days_step=20)
    fluxes1 = np.concatenate( (fluxes1, out_data.get('fluxes1')) )
    fluxes2 = np.concatenate( (fluxes2, out_data.get('fluxes2')) )
    SymH = np.concatenate( (SymH, out_data.get('SymH')) )
    print('Done: %s' % (date_current))

N_SymH = 40
N_fluxes = 100

SymH_bins = fitFlows.get_equinumeros_subests_on_param('SymH', numOfBins=N_SymH, R=4, date_start_str=date_start.strftime('%Y%m%d'), date_end_str=date_end.strftime('%Y%m%d'))
indsH = np.digitize(SymH,SymH_bins)
fluxes_bins = np.logspace(-3,8,N_fluxes+1)

Done: 2012-11-06
Done: 2013-01-05
Done: 2013-03-06
Done: 2013-05-05
Done: 2013-07-04
Done: 2013-09-02
Done: 2013-11-01
Done: 2013-12-31
Done: 2014-03-01
Done: 2014-04-30
Done: 2014-06-29
Done: 2014-08-28
Done: 2014-10-27
Done: 2014-12-26
Done: 2015-02-24
Done: 2015-04-25
Done: 2015-06-24
Done: 2015-08-23
Done: 2015-10-22
Done: 2015-12-21
Done: 2016-02-19
Done: 2016-04-19
Done: 2016-06-18
Done: 2016-08-17
Done: 2016-10-16
Done: 2016-12-15
Done: 2017-02-13
Done: 2017-04-14
Done: 2017-06-13
Done: 2017-08-12
Done: 2017-10-11
Done: 2017-12-10
Done: 2018-02-08
Done: 2018-04-09
Done: 2018-06-08
Done: 2018-08-07
Done: 2018-10-06
Done: 2018-12-05
Done: 2019-02-03
Done: 2019-04-04
Done: 2019-06-03
Done: 2019-08-02
Done: 2019-10-01
Done: 2019-11-30


In [6]:
flx1 = fluxes1[indsH==(3)]
flux_hist, bins = np.histogram(flx1, bins=fluxes_bins)
flx1.shape

(501391, 25)

In [10]:
N_energy = 25

In [11]:
flux_hist = np.zeros((N_fluxes, N_energy, N_SymH))

In [7]:
fluxes1[indsH==(3), 5]

array([ 1765.85601997,  1784.17161874, 10767.7854751 , ...,
        6715.80186964,  5579.97342929,  6273.07935348])

In [15]:
for i in range(N_SymH):
    print(i/N_SymH)
    for j in range(N_energy):
        flux_hist[:,j,i], bins = np.histogram(fluxes1[indsH==(i+1),j], bins=fluxes_bins)


0.0
0.025
0.05
0.075
0.1
0.125
0.15
0.175
0.2
0.225
0.25
0.275
0.3
0.325
0.35
0.375
0.4
0.425
0.45
0.475
0.5
0.525
0.55
0.575
0.6
0.625
0.65
0.675
0.7
0.725
0.75
0.775
0.8
0.825
0.85
0.875
0.9
0.925
0.95
0.975


In [34]:
from Globals import *
dirname = database + "data_paper\\figure4\\"

for i in range(N_energy):
    np.savetxt(dirname+'flux_hist_'+str(i)+'.txt', flux_hist[:,i,:])

In [49]:
for i in range(fluxes1.shape[1]):
    hist, bin_edges = np.histogram(SymH, bins=SymH_bins, weights=fluxes1[:,i])

In [23]:
np.histogram(SymH, bins=SymH_bins, weights=fluxes1[:,3])

(array([1.03437567e+11, 8.79413375e+10, 8.26394849e+10, 7.77817137e+10,
        6.95524533e+10, 6.21096548e+10, 6.87631918e+10, 5.81021241e+10,
        5.70821330e+10, 4.18595291e+10, 7.09944787e+10, 5.30564987e+10,
        5.11117918e+10, 5.23148047e+10, 5.04414174e+10, 2.52516597e+10,
        5.25629735e+10, 5.56097310e+10, 4.60956546e+10, 1.17961371e+10,
        5.66137543e+10, 6.00091606e+10, 4.08874619e+10, 1.60972622e+10,
        6.17208123e+10, 5.20032198e+10, 9.65874028e+09, 5.89865128e+10,
        5.41660953e+10, 3.87828518e+10, 3.89682992e+10, 1.71068907e+10,
        4.70679741e+10, 4.02741004e+10, 3.66488334e+10, 5.46417600e+10,
        3.13984136e+10, 4.52767236e+10, 3.69288242e+10, 4.06227256e+10]),
 array([-2.32972094e+02, -4.79922686e+01, -3.77087507e+01, -3.20000000e+01,
        -2.80154482e+01, -2.52369142e+01, -2.30000000e+01, -2.11493601e+01,
        -1.99113775e+01, -1.80000000e+01, -1.70000000e+01, -1.59440004e+01,
        -1.47185908e+01, -1.36795359e+01, -1.27754

In [19]:
mageis_vars = pyspedas.rbsp.mageis(['2015-01-01/00:00:00', '2015-01-01/01:00:00'], probe='a', level='l3', rel='rel04', notplot=True)
mageis_vars.keys()

24-Dec-21 02:43:54: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/rbsp/rbspa/l3/ect/mageis/sectors/rel04/2015/
24-Dec-21 02:43:55: File is current: D:\SPEDAS files\rbsp\rbspa/l3/ect/mageis/sectors/rel04/2015/rbspa_rel04_ect-mageis-l3_20150101_v8.1.0.cdf
24-Dec-21 02:43:56: d:\work_Pavel\7. PlasmaFlowsForecast\python\env\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),



dict_keys(['FEDU', 'FEDU_ERROR', 'FEDU_CORR', 'FEDU_CORR_ERROR', 'FPDU', 'FPDU_pix1', 'FPDU_pix2', 'L_star', 'L', 'I', 'B_Calc', 'B_Eq', 'MLT', 'MLAT', 'Position', 'LstarVsAlpha', 'LstarVsAlpha_Alpha'])

In [ ]:
mageis_vars = pyspedas.rbsp.mageis(['2015-01-01/00:00:00', '2015-01-01/01:00:00'], probe='a', level='l3', rel='rel04', notplot=True)
energy = mageis_vars.get('FEDU').get('v2') 
dep = mageis_vars.get('FEDU').get('v2') 

In [35]:
dirname = database + "data_paper\\figure4\\"
energy_bins = np.loadtxt(dirname+'energy_bins.txt')
fluxes_bins = np.loadtxt(dirname+'fluxes_bins.txt')
SymH_bins   = np.loadtxt(dirname+'SymH_bins.txt')

N_energy = len(energy_bins) - 1
N_fluxes = len(fluxes_bins) - 1
N_SymH   = len(SymH_bins) - 1
    
flux_hist = np.zeros((N_fluxes, N_SymH, N_energy))
dirname = database + "data_paper\\figure4\\flux_hist\\"
for i in range(N_energy):
    flux_hist[:,:,i] = np.loadtxt(dirname+'flux_hist_'+str(i)+'.txt')

In [36]:
flux_hist.shape

(100, 40, 25)